# logs

data

https://www.usenix.org/cfdr-data

https://www.usenix.org/sites/default/files/4366-0809181018.tar.gz


In [ ]:

import sys
import tarfile
from pathlib import Path
from calendar import month_abbr

import pandas as pd
import plotly.express as px
import requests

sys.path.append("logparser")

from logparser.Spell import Spell


In [ ]:

url = "https://www.usenix.org/sites/default/files/4366-0809181018.tar.gz"

with requests.get(url, stream=True) as r:
    with open("4366-0809181018.tar.gz", 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)


In [ ]:

with tarfile.open("4366-0809181018.tar.gz", "r") as f:
    f.extractall()


In [ ]:

parser = Spell.LogParser(log_format="\[<Timestamp>\]:<Content>", indir="0809181018", outdir=".", tau=0.5)
parser.parse("bnd.log")


df = pd.read_csv(next(Path().glob("*_structured.csv")))

(
    px.histogram(
        df
        .assign(
            Timestamp=lambda x0:
            x0["Timestamp"]
            .str.extract(r"\S+\s+(?P<m>\S+)\s+(?P<d>\S+)\s+(?P<t>\S+)\s+(?P<y>\S+)")
            .replace(dict(zip(month_abbr, range(13))))
            .assign(m=lambda x: x["m"].astype(str).str.rjust(2, "0"))
            .assign(d=lambda x: x["d"].astype(str).str.rjust(2, "0"))
            .pipe(lambda x: pd.to_datetime(x["y"] + "-" + x["m"] + "-" + x["d"] + " " + x["t"])))
        [["Timestamp", "EventTemplate"]],
        x="Timestamp",
        color="EventTemplate",
        nbins=288)
    .update_layout(height=500, legend=dict(y=-0.5, orientation="h"), title="logs")
    .show()
)
